In [41]:
import numpy as np
import pandas as pd
from scipy.stats import norm, chi2_contingency

In [ ]:
def dprime(hit_rate, fa_rate, n_trials=100):
    """Compute d′ from hit and false alarm rates with correction for 0/1."""
    # Correction to avoid inf z-scores
    if hit_rate == 1: hit_rate = 1 - 1/(2*n_trials)
    if hit_rate == 0: hit_rate = 1/(2*n_trials)
    if fa_rate == 1:  fa_rate  = 1 - 1/(2*n_trials)
    if fa_rate == 0:  fa_rate  = 1/(2*n_trials)
    return norm.ppf(hit_rate) - norm.ppf(fa_rate)

def chi_square_from_rates(hit_rate, fa_rate, n_trials=112):
    """Run χ² test comparing detection vs. guessing."""
    n_signal = n_noise = n_trials / 2
    obs = np.array([
        [hit_rate * n_signal, (1 - hit_rate) * n_signal],
        [fa_rate  * n_noise,  (1 - fa_rate)  * n_noise]
    ])
    chi2, p, _, _ = chi2_contingency(obs)
    return chi2, p

def signif_marker(p):
    """Return asterisk for significance."""
    if p < 0.001: return "**"
    elif p < 0.05: return "*"
    else: return ""

# Example data from dahaene et al.)
prime_durations = [0, 29, 43, 57, 114, 200]
hit_rates = [0.286, 0.402, 0.491, 0.464, 0.786, 0.955]
fa_rates  = [0.348, 0.321, 0.411, 0.304, 0.286, 0.161]

# Compute d′ and χ² for each condition
rows = []
for dur, h, f in zip(prime_durations, hit_rates, fa_rates):
    dp = dprime(h, f, n_trials=112)
    chi2, p = chi_square_from_rates(h, f, n_trials=112)
    rows.append({
        "Prime duration (ms)": dur,
        "Hit rate (%)": f"{h*100:.1f}",
        "False alarms (%)": f"{f*100:.1f}",
        "d′": f"{dp:.2f}{signif_marker(p)}",
        "χ²": f"{chi2:.2f}",
        "p": f"{p:.4g}"
    })

df = pd.DataFrame(rows)
print(df.to_string(index=False))
